<a href="https://colab.research.google.com/github/omer-re/will_it_fit/blob/main/will_it_fit_in_elevator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📌 Enable inline plotting
%matplotlib inline

# 📌 Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from scipy.spatial.transform import Rotation as R
from itertools import product
import ipywidgets as widgets
from IPython.display import display

# 📌 Suppress font glyph warnings (optional)
import warnings

warnings.filterwarnings("ignore")


# 📌 Utility: Draw a 3D box in space
def draw_box(ax, origin, size, color, alpha=0.2, label=None):
    ox, oy, oz = origin
    l, w, h = size
    x = [ox, ox + l]
    y = [oy, oy + w]
    z = [oz, oz + h]

    vertices = [
        [x[0], y[0], z[0]],
        [x[1], y[0], z[0]],
        [x[1], y[1], z[0]],
        [x[0], y[1], z[0]],
        [x[0], y[0], z[1]],
        [x[1], y[0], z[1]],
        [x[1], y[1], z[1]],
        [x[0], y[1], z[1]],
    ]

    faces = [
        [vertices[0], vertices[1], vertices[2], vertices[3]],  # bottom
        [vertices[4], vertices[5], vertices[6], vertices[7]],  # top
        [vertices[0], vertices[1], vertices[5], vertices[4]],  # front
        [vertices[2], vertices[3], vertices[7], vertices[6]],  # back
        [vertices[1], vertices[2], vertices[6], vertices[5]],  # right
        [vertices[3], vertices[0], vertices[4], vertices[7]],  # left
    ]

    ax.add_collection3d(Poly3DCollection(faces, facecolors=color, edgecolors='black', alpha=alpha))

    if label:
        ax.text(ox + l / 2, oy + w / 2, oz + h + 5, label, color='black', ha='center')


# 📌 Check if all rotated corners are inside the elevator volume
def check_fit_in_elevator(corners, elevator_dims):
    corners = np.array(corners)
    return (
            (corners[:, 0] >= 0).all() and (corners[:, 0] <= elevator_dims[0]).all() and
            (corners[:, 1] >= 0).all() and (corners[:, 1] <= elevator_dims[1]).all() and
            (corners[:, 2] >= 0).all() and (corners[:, 2] <= elevator_dims[2]).all()
    )


# 📌 Check if projection of rotated box can pass through door opening (YZ projection)
def check_fit_through_door(rotated_corners, door_dims):
    projection = rotated_corners[:, [1, 2]]  # Project onto Y-Z plane
    min_corner = np.min(projection, axis=0)
    max_corner = np.max(projection, axis=0)
    width_required, height_required = max_corner - min_corner

    door_width, door_height = door_dims
    return width_required <= door_width and height_required <= door_height


# 📌 Main function to test and visualize
def find_valid_fit_with_door_constraint(box_small, box_large, door_dims):
    l, w, h = box_small
    corners = np.array([
        [0, 0, 0],
        [l, 0, 0],
        [l, w, 0],
        [0, w, 0],
        [0, 0, h],
        [l, 0, h],
        [l, w, h],
        [0, w, h]
    ])
    centroid = np.mean(corners, axis=0)
    corners_centered = corners - centroid

    best_fit = None
    # 🔁 Optimization: Use coarse 10° steps for fast testing
    for yaw, pitch, roll in product(range(0, 91, 10), repeat=3):
        rot = R.from_euler('zyx', [yaw, pitch, roll], degrees=True)
        rotated = rot.apply(corners_centered) + np.array(box_large) / 2

        if check_fit_in_elevator(rotated, box_large) and check_fit_through_door(rotated, door_dims):
            best_fit = (rotated, yaw, pitch, roll)
            break

    fig = plt.figure(figsize=(12, 9))
    ax = fig.add_subplot(111, projection='3d')
    draw_box(ax, origin=(0, 0, 0), size=box_large, color='blue', alpha=0.1, label='Elevator')

    if best_fit:
        rotated_corners, yaw, pitch, roll = best_fit
        faces = [
            [0, 1, 2, 3],
            [4, 5, 6, 7],
            [0, 1, 5, 4],
            [2, 3, 7, 6],
            [1, 2, 6, 5],
            [3, 0, 4, 7]
        ]
        sofa_faces = [[rotated_corners[i] for i in face] for face in faces]
        ax.add_collection3d(
            Poly3DCollection(sofa_faces, facecolors='green', edgecolors='black', linewidths=1, alpha=0.5))
        ax.text(*np.mean(rotated_corners, axis=0), f"Yaw={yaw}°, Pitch={pitch}°, Roll={roll}°", color='black',
                ha='center')
        ax.set_title("✅ Valid Fit Through Door and Elevator")
    else:
        ax.set_title("❌ No Fit Found (Including Door Constraints)")

    ax.set_xlabel('Length (X)')
    ax.set_ylabel('Width (Y)')
    ax.set_zlabel('Height (Z)')
    max_range = max(box_large)
    ax.set_xlim(0, max_range)
    ax.set_ylim(0, max_range)
    ax.set_zlim(0, max_range)
    plt.tight_layout()
    plt.show()

    return best_fit is not None


# 📌 UI: Create interactive widgets for user input
sofa_length = widgets.IntSlider(value=270, min=50, max=400, step=5, description='Sofa Length:')
sofa_width = widgets.IntSlider(value=90, min=40, max=200, step=5, description='Sofa Width:')
sofa_height = widgets.IntSlider(value=50, min=5, max=150, step=5, description='Sofa Height:')

elevator_length = widgets.IntSlider(value=238, min=100, max=300, step=5, description='Elevator Len:')
elevator_width = widgets.IntSlider(value=100, min=80, max=200, step=5, description='Elevator Wid:')
elevator_height = widgets.IntSlider(value=215, min=150, max=300, step=5, description='Elevator Hei:')

door_width = widgets.IntSlider(value=100, min=60, max=150, step=5, description='Door Width:')
door_height = widgets.IntSlider(value=215, min=150, max=300, step=5, description='Door Height:')

run_button = widgets.Button(description="Run Fit Check")


# 📌 Callback function
def on_button_click(b):
    sofa = [sofa_length.value, sofa_width.value, sofa_height.value]
    elevator = [elevator_length.value, elevator_width.value, elevator_height.value]
    door = [door_width.value, door_height.value]

    print(f"🛋️ Sofa: {sofa}")
    print(f"🏢 Elevator: {elevator}")
    print(f"🚪 Door: {door}")
    find_valid_fit_with_door_constraint(sofa, elevator, door)


run_button.on_click(on_button_click)

# 📌 Display all widgets
display(
    widgets.VBox([
        widgets.HTML("<h3>📐 Sofa Dimensions</h3>"),
        sofa_length, sofa_width, sofa_height,
        widgets.HTML("<h3>📦 Elevator Interior Dimensions</h3>"),
        elevator_length, elevator_width, elevator_height,
        widgets.HTML("<h3>🚪 Elevator Door Dimensions</h3>"),
        door_width, door_height,
        run_button
    ])
)


In [ ]:

#📌 Enable inline plotting

%matplotlib inline

#📌 Import required libraries

import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from scipy.spatial.transform import Rotation as R
import ipywidgets as widgets
from IPython.display import display

#📌 3D export/view

import trimesh
import plotly.graph_objects as go

#📌 Suppress warnings

import warnings

warnings.filterwarnings("ignore")


#📌 Utility: Draw a 3D box in space

def draw_box(ax, origin, size, color, alpha=0.2, label=None):
    ox, oy, oz = origin
    l, w, h = size
    x = [ox, ox + l]
    y = [oy, oy + w]
    z = [oz, oz + h]
    vertices = [[x[0], y[0], z[0]], [x[1], y[0], z[0]], [x[1], y[1], z[0]], [x[0], y[1], z[0]], [x[0], y[0], z[1]],
                [x[1], y[0], z[1]], [x[1], y[1], z[1]], [x[0], y[1], z[1]], ]
    faces = [[vertices[0], vertices[1], vertices[2], vertices[3]], [vertices[4], vertices[5], vertices[6], vertices[7]],
             [vertices[0], vertices[1], vertices[5], vertices[4]], [vertices[2], vertices[3], vertices[7], vertices[6]],
             [vertices[1], vertices[2], vertices[6], vertices[5]], [vertices[3], vertices[0], vertices[4], vertices[7]], ]
    ax.add_collection3d(Poly3DCollection(faces, facecolors=color, edgecolors='black', alpha=alpha))
    if label: ax.text(ox + l / 2, oy + w / 2, oz + h + 5, label, ha='center')


#📌 Fit checks (unchanged)

def check_fit_in_elevator(corners, dims):
    c = np.array(corners)
    return ((c >= 0) & (c <= dims)).all()


def check_fit_through_door(corners, door_dims):
    proj = corners[:, [1, 2]]

    mn, mx = proj.min(axis=0), proj.max(axis=0)
    req = mx - mn
    return (req[0] <= door_dims[0]) and (req[1] <= door_dims[1])


#📌 Penalty for closest fit (unchanged)

def penalty_for(corners, elev_dims, door_dims):
    mn, mx = corners.min(axis=0), corners.max(axis=0)

    pen_e = sum(max(0, -mn[i]) + max(0, mx[i] - elev_dims[i]) for i in range(3))
    proj = corners[:, [1, 2]]
    mn2, mx2 = proj.min(axis=0), proj.max(axis=0)
    req = mx2 - mn2
    pen_d = max(0, req[0] - door_dims[0]) + max(0, req[1] - door_dims[1])
    return pen_e + pen_d


#📌 Export & interactive view: places two axis-aligned boxes

def export_and_view_scene(box_small, box_large, rotated_corners, yaw, pitch, roll):
    l, w, h = box_small
    filename = f"elevator_sofa_scene_{l}{w}{h}.stl"
    os.makedirs(os.path.dirname(filename) or '.', exist_ok=True)

    # Elevator mesh at origin
    elev_mesh = trimesh.creation.box(extents=box_large)
    elev_mesh.apply_translation(np.array(box_large) / 2)

    # Sofa mesh placed next to elevator (gap = 10 units)
    sofa_mesh = trimesh.creation.box(extents=box_small)
    gap = 10
    # Position sofa along X axis just outside elevator
    sofa_origin = np.array([box_large[0] + gap, 0, 0])
    sofa_mesh.apply_translation(sofa_origin + np.array(box_small) / 2)

    # Combine and export two separate bodies
    scene = trimesh.Scene([elev_mesh, sofa_mesh])
    scene.export(filename)
    print(f"✅ Exported STL to {filename}")


    # Interactive Plotly view
    def mesh3d(m, color, opacity):
        v, f = m.vertices, m.faces
        return go.Mesh3d(
            x=v[:, 0], y=v[:, 1], z=v[:, 2],
            i=f[:, 0], j=f[:, 1], k=f[:, 2],
            opacity=opacity, color=color
        )


        fig = go.Figure([
            mesh3d(elev_mesh, 'lightblue', 0.2),
            mesh3d(sofa_mesh, 'green', 0.5)
        ])
        fig.update_layout(
            scene=dict(aspectmode='data', xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),
            title=f"3D Preview — Elevator & Sofa (unrotated boxes)"
        )
        fig.show()


#📌 Main: search + preview + export (unchanged orientation logic)

def find_valid_fit_with_door_constraint(box_small, box_large, door_dims):
    l, w, h = box_small
    corners = np.array([[0, 0, 0], [l, 0, 0], [l, w, 0], [0, w, 0], [0, 0, h], [l, 0, h], [l, w, h], [0, w, h]])
    centered = corners - corners.mean(axis=0)

    best_valid = None
    best_cand = (None, float('inf'), 0, 0, 0)

    for yaw in range(0, 91, 10):
        for pitch in range(0, 91, 10):
            for roll in range(0, 91, 10):
                rotated = R.from_euler('zyx', [yaw, pitch, roll], degrees=True).apply(centered) + np.array(box_large) / 2
                pen = penalty_for(rotated, box_large, door_dims)
                if pen < best_cand[1]:
                    best_cand = (rotated, pen, yaw, pitch, roll)
                if pen == 0:
                    best_valid = (rotated, yaw, pitch, roll)
                    break
            if best_valid: break
        if best_valid: break

    if best_valid:
        rc, yaw, pitch, roll = best_valid
        title = "✅ Valid Fit Found"
    else:
        rc, _, yaw, pitch, roll = best_cand
        title = "❌ No Valid Fit — showing closest orientation"

    # 3D Matplotlib preview
    fig = plt.figure(figsize=(12, 9))
    ax = fig.add_subplot(111, projection='3d')
    draw_box(ax, (0, 0, 0), box_large, 'blue', alpha=0.1, label='Elevator')
    draw_box(ax, (box_large[0] + 10, 0, 0), box_small, 'green', alpha=0.5, label='Sofa')
    ax.set_title(title)
    ax.set_xlabel('X');
    ax.set_ylabel('Y');
    ax.set_zlabel('Z')
    mr = box_large[0] + box_small[0] + 10
    ax.set_xlim(0, mr);
    ax.set_ylim(0, max(box_large[1], box_small[1]));
    ax.set_zlim(0, max(box_large[2], box_small[2]))
    plt.tight_layout();
    plt.show()

    # Export & interactive view
    export_and_view_scene(box_small, box_large, None, None, None, None)

#📌 Interactive UI: sliders trigger live update

sofa_l = widgets.IntSlider(value=270, min=50, max=400, step=5, description='Sofa L:')
sofa_w = widgets.IntSlider(value=90, min=40, max=200, step=5, description='Sofa W:')
sofa_h = widgets.IntSlider(value=50, min=5, max=150, step=5, description='Sofa H:')

elev_l = widgets.IntSlider(value=238, min=100, max=300, step=5, description='Elev L:')
elev_w = widgets.IntSlider(value=100, min=80, max=200, step=5, description='Elev W:')
elev_h = widgets.IntSlider(value=215, min=150, max=300, step=5, description='Elev H:')

door_w = widgets.IntSlider(value=100, min=60, max=150, step=5, description='Door W:')
door_h = widgets.IntSlider(value=215, min=150, max=300, step=5, description='Door H:')

out = widgets.interactive_output(
    lambda sl, sw, sh, el, ew, eh, dw, dh: find_valid_fit_with_door_constraint([sl, sw, sh], [el, ew, eh], [dw, dh]),
    {'sl': sofa_l, 'sw': sofa_w, 'sh': sofa_h, 'el': elev_l, 'ew': elev_w, 'eh': elev_h, 'dw': door_w, 'dh': door_h})

controls = widgets.VBox(
    [widgets.HTML("<h3>Sofa Dimensions</h3>"), sofa_l, sofa_w, sofa_h, widgets.HTML("<h3>Elevator Dimensions</h3>"),
     elev_l, elev_w, elev_h, widgets.HTML("<h3>Door Dimensions</h3>"), door_w, door_h])

display(controls, out)





In [ ]:
!pip install trimesh

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from scipy.spatial.transform import Rotation as R
from itertools import product
from matplotlib.animation import FuncAnimation, FFMpegWriter

# ─── Helper functions ────────────────────────────────────────────────────────

def check_fit_in_elevator(corners, dims):
    c = np.array(corners)
    return (
        (c >= 0).all() and
        (c[:,0] <= dims[0]).all() and
        (c[:,1] <= dims[1]).all() and
        (c[:,2] <= dims[2]).all()
    )

def check_fit_through_door(corners, door_dims):
    proj = corners[:, [1,2]]
    mn, mx = proj.min(axis=0), proj.max(axis=0)
    req = mx - mn
    return req[0] <= door_dims[0] and req[1] <= door_dims[1]

def find_solution(box_small, box_large, door_dims, step_deg=10, roll_step=5):
    """
    Scan yaw/pitch/roll grid; return the first (yaw, pitch, roll, rotated_corners)
    that fits both elevator and door constraints.
    """
    angles = list(product(
        range(0, 91, step_deg),
        range(0, 91, step_deg),
        range(0, 91, roll_step)
    ))
    l, w, h = box_small
    corners = np.array([
        [0,0,0],[l,0,0],[l,w,0],[0,w,0],
        [0,0,h],[l,0,h],[l,w,h],[0,w,h]
    ])
    center = corners.mean(axis=0)
    for yaw, pitch, roll in angles:
        rot = R.from_euler('zyx', [yaw, pitch, roll], degrees=True)
        rotated = rot.apply(corners - center) + np.array(box_large) / 2
        if check_fit_in_elevator(rotated, box_large) and \
           check_fit_through_door(rotated, door_dims):
            return yaw, pitch, roll, rotated
    return None, None, None, None

def draw_box(ax, origin, size, color, alpha=0.2):
    ox, oy, oz = origin
    l, w, h = size
    verts = np.array([
        [ox,     oy,     oz],
        [ox+l,   oy,     oz],
        [ox+l,   oy+w,   oz],
        [ox,     oy+w,   oz],
        [ox,     oy,     oz+h],
        [ox+l,   oy,     oz+h],
        [ox+l,   oy+w,   oz+h],
        [ox,     oy+w,   oz+h]
    ])
    faces_idx = [
        [0,1,2,3], [4,5,6,7],
        [0,1,5,4], [2,3,7,6],
        [1,2,6,5], [3,0,4,7]
    ]
    faces = [[verts[i] for i in face] for face in faces_idx]
    ax.add_collection3d(Poly3DCollection(faces, facecolors=color, edgecolors='k', alpha=alpha))

# ─── Main Matplotlib exporter ────────────────────────────────────────────────

def export_orbit_mp4_matplotlib(
    box_small, box_large, door_dims,
    step_deg=10, roll_step=5,
    n_frames=60, filename='orbit_demo.mp4', fps=10
):
    yaw, pitch, roll, sofa_verts = find_solution(
        box_small, box_large, door_dims, step_deg, roll_step
    )
    if sofa_verts is None:
        print("❌ No valid fit found.")
        return False

    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111, projection='3d')
    max_range = max(box_large)

    def update(frame):
        ax.cla()
        # Draw static elevator box
        draw_box(ax, (0,0,0), box_large, color='lightblue', alpha=0.1)
        # Draw sofa in found orientation
        faces_idx = [
            [0,1,2,3], [4,5,6,7],
            [0,1,5,4], [2,3,7,6],
            [1,2,6,5], [3,0,4,7]
        ]
        sofa_faces = [[sofa_verts[i] for i in face] for face in faces_idx]
        ax.add_collection3d(Poly3DCollection(
            sofa_faces, facecolors='green', edgecolors='k', alpha=0.6
        ))
        # Fix axes
        ax.set_xlim(0, max_range)
        ax.set_ylim(0, max_range)
        ax.set_zlim(0, max_range)
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        # Rotate view
        azim = frame * (360 / n_frames)
        ax.view_init(elev=30, azim=azim)
        ax.set_title(f"Yaw={yaw}°, Pitch={pitch}°, Roll={roll}°", fontweight='bold')

    anim = FuncAnimation(fig, update, frames=n_frames, interval=1000/fps)
    writer = FFMpegWriter(fps=fps)
    anim.save(filename, writer=writer)
    plt.close(fig)
    print(f"✅ Saved MP4 animation: {filename}")
    return True

# ─── Example usage ──────────────────────────────────────────────────────────

if __name__ == "__main__":
    sofa_dims = [270, 70, 25]    # Updated sofa dimensions
    elevator  = [238, 100, 215]  # Elevator interior dimensions
    door_dims = [100, 215]       # Door opening

    export_orbit_mp4_matplotlib(
        sofa_dims,
        elevator,
        door_dims,
        step_deg=10,
        roll_step=5,
        n_frames=60,
        filename='orbit_demo.mp4',
        fps=10
    )


✅ Saved MP4 animation: orbit_demo.mp4
